<a href="https://colab.research.google.com/github/jakes1811/Smart-Waste-Segregation-System/blob/main/WebCam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load trained model
model = tf.keras.models.load_model("C:\\Users\\jenif\\Downloads\\waste_classifier_mobilenet.h5")
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
labels = ['Cardboard', 'E waste', 'Glass','Metal', 'Paper', 'Plastic']

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
         # Preprocess frame
    img = cv2.resize(frame, (128,128))

    # Convert to grayscale and apply histogram equalization
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    equalized = cv2.equalizeHist(gray)
    # Convert back to 3-channel image
    img = cv2.cvtColor(equalized, cv2.COLOR_GRAY2BGR)
    img = img.astype("float32")/255.0
    img = np.expand_dims(img, axis=0)
    from tensorflow.keras.preprocessing.image import ImageDataGenerator
    datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
    # Predict
    predictions = model.predict(img)
    predicted_class= np.argmax(predictions)
    confidence = np.max(predictions)

    label = f"{labels[predicted_class]}({confidence:.2f})"

    # Display result
    cv2.putText(frame, f'Waste: {label}', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Waste Classification", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Verify the class order from your training process
print(train_generator.class_indices)